In [48]:
import sys
import pandas as pd
import scib
print(f"Python version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"scib version: {scib.__version__}")

Python version: 3.10.15 | packaged by conda-forge | (main, Sep 20 2024, 16:37:05) [GCC 13.3.0]
pandas version: 2.2.2
scib version: 1.1.5


In [49]:
import os
os.environ['R_HOME'] = '/home/data/t090402/miniforge3/envs/scib/lib/R'

In [50]:
import scanpy as sc
import pandas as pd
import os
import sys

# Required step before importing scib
RHOME = os.path.join(sys.executable.split('/bin/')[0],'lib/R')
from rpy2.rinterface_lib import openrlib
openrlib.R_HOME = RHOME
import scib

In [51]:
adata = sc.read_h5ad('/home/data/t090402/data/PBMC/h5ad/all.h5ad')
adata.X = adata.X.tocsr()

In [52]:
integration_result = {}
bench_result = {}
BATCH_KEY = 'sample'
CELLTYPE_KEY = 'multi_annotation'

In [53]:
adata

AnnData object with n_obs × n_vars = 168716 × 228328
    obs: 'sample', 'multi_annotation'
    var: 'name'

In [54]:
# Total-count normalize (library-size correct) to 10,000 reads/cell
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)

# log transform the data.
sc.pp.log1p(adata)

# identify highly variable genes.
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

# keep only highly variable genes:
adata = adata[:, adata.var['highly_variable']]

/home/data/t090402/miniforge3/envs/scib/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:583: DeprecationWarning: The specified parameters ('counts_per_cell_after', 'counts_per_cell') are no longer positional. Please specify them like `counts_per_cell_after=10000.0`
  normalize_per_cell(adata.X, counts_per_cell_after, counts_per_cell)


In [55]:
# Run PCA prior to run Harmony
sc.tl.pca(adata)

/home/data/t090402/miniforge3/envs/scib/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:317: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm[key_obsm] = X_pca


In [56]:
adata

AnnData object with n_obs × n_vars = 168716 × 15751
    obs: 'sample', 'multi_annotation', 'n_counts'
    var: 'name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'

In [57]:
eval_metrics = {
        'isolated_labels_asw_': True,
        'silhouette_':True,
        'hvg_score_':True,
        'graph_conn_':True,
        'pcr_':True,
        'isolated_labels_f1_':True,
        'nmi_':True,
        'ari_':True,
        'cell_cycle_':False,
        'kBET_':True,
        'ilisi_':True,
        'clisi_':True
}

In [58]:
adata

AnnData object with n_obs × n_vars = 168716 × 15751
    obs: 'sample', 'multi_annotation', 'n_counts'
    var: 'name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'

In [59]:
adata

AnnData object with n_obs × n_vars = 168716 × 15751
    obs: 'sample', 'multi_annotation', 'n_counts'
    var: 'name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'

In [ ]:
# Harmony runs through R so:
sc.external.pp.harmony_integrate(adata, BATCH_KEY)
integration_result['harmony'] = adata
sc.pp.neighbors(integration_result['harmony'])

2024-09-30 19:53:59,251 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
OpenBLAS warning: precompiled NUM_THREADS exceeded, adding auxiliary array for thread metadata.
To avoid this warning, please rebuild your copy of OpenBLAS with a larger NUM_THREADS setting
or set the environment variable OPENBLAS_NUM_THREADS to 64 or lower


In [ ]:
integration_result['harmony']

In [62]:
# Compute the evaluation metrics for Harmony
bench_result['harmony'] = scib.me.metrics(
        adata,
        integration_result['harmony'],
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY, 
        verbose = True,
        embed = 'X_harmony',
        organism = 'human',
        **eval_metrics
)

/home/data/t090402/miniforge3/envs/scib/lib/python3.10/site-packages/scib/metrics/metrics.py:293: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  res_max, nmi_max, nmi_all = opt_louvain(


Clustering...
resolution: 0.1, nmi: 0.5251834732394216
resolution: 0.2, nmi: 0.5458345741039923
resolution: 0.3, nmi: 0.5473299977122658
resolution: 0.4, nmi: 0.5450727664082351
resolution: 0.5, nmi: 0.551295372400717
resolution: 0.6, nmi: 0.5410868966844717
resolution: 0.7, nmi: 0.5424371160134206
resolution: 0.8, nmi: 0.5436091416956094
resolution: 0.9, nmi: 0.5434445883907676
resolution: 1.0, nmi: 0.5438844455024197
resolution: 1.1, nmi: 0.5507901111926689
resolution: 1.2, nmi: 0.5493772421530554
resolution: 1.3, nmi: 0.5493069079008343
resolution: 1.4, nmi: 0.543441795901938
resolution: 1.5, nmi: 0.5414434780586771
resolution: 1.6, nmi: 0.5425080194025808
resolution: 1.7, nmi: 0.5399043049596598
resolution: 1.8, nmi: 0.543253783385494
resolution: 1.9, nmi: 0.5430111387601767
resolution: 2.0, nmi: 0.5430378335705127
optimised clustering against multi_annotation
optimal cluster resolution: 0.5
optimal score: 0.551295372400717
NMI...
ARI...
Silhouette score...
KeysView(AxisArrays with

KeyError: 'X_harmony not in obsm'

In [ ]:
bench_result['harmony'].rename(columns = {0 : 'harmony'}, inplace = True)
bench_result['harmony']

In [ ]:
adata

In [ ]:
# Run bbknn
integration_result['bbknn'] = scib.integration.bbknn(adata, BATCH_KEY)
sc.pp.neighbors(integration_result['bbknn'])
# Eval bbknn
bench_result['bbknn'] = scib.me.metrics(
        adata,
        integration_result['bbknn'], 
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True, embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
# Rename column
bench_result['bbknn'].rename(columns = {0 : 'bbknn'}, inplace = True)
bench_result['bbknn']

In [ ]:
# Run scanorama
integration_result['scanorama'] = scib.integration.scanorama(adata, BATCH_KEY)
integration_result['scanorama'].obs[BATCH_KEY] = adata.obs[BATCH_KEY]
integration_result['scanorama'].obs[CELLTYPE_KEY] = adata.obs[CELLTYPE_KEY].astype('category')

# Bench scanorama
bench_result['scanorama'] = scib.me.metrics(
        adata,
        integration_result['scanorama'], 
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True,
        embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
bench_result['scanorama'].rename(columns = {0 : 'scanorama'}, inplace = True)
bench_result['scanorama']

In [ ]:
# Run combat
integration_result['combat'] = scib.integration.combat(adata, BATCH_KEY)
sc.pp.neighbors(integration_result['combat'])
# Bench combat
bench_result['combat'] = scib.me.metrics(
        adata,
        integration_result['combat'],
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True,
        embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
bench_result['combat'].rename(columns = {0 : 'combat'}, inplace = True)
bench_result['combat']

In [ ]:
# Run saucie
integration_result['saucie'] = scib.integration.combat(adata, BATCH_KEY)
sc.pp.neighbors(integration_result['saucie'])
# Bench saucie
bench_result['saucie'] = scib.me.metrics(
        adata,
        integration_result['saucie'],
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True,
        embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
# Rename column
bench_result['saucie'].rename(columns = {0 : 'saucie'}, inplace = True)
bench_result['saucie']

In [ ]:
# Run trvae
integration_result['trvae'] = scib.integration.combat(adata, BATCH_KEY)
sc.pp.neighbors(integration_result['trvae'])
# Bench saucie
bench_result['trvae'] = scib.me.metrics(
        adata,
        integration_result['trvae'],
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True,
        embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
# Rename column
bench_result['trvae'].rename(columns = {0 : 'trvae'}, inplace = True)
bench_result['trvae']

In [ ]:
# Run tvraep
integration_result['trvaep'] = scib.integration.combat(adata, BATCH_KEY)
sc.pp.neighbors(integration_result['trvaep'])
# Bench saucie
bench_result['trvaep'] = scib.me.metrics(
        adata,
        integration_result['trvaep'],
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True,
        embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
# Rename column
bench_result['trvaep'].rename(columns = {0 : 'trvaep'}, inplace = True)
bench_result['trvaep']

In [ ]:
# Run scgen
integration_result['scgen'] = scib.integration.combat(adata, BATCH_KEY)
sc.pp.neighbors(integration_result['scgen'])
# Bench saucie
bench_result['scgen'] = scib.me.metrics(
        adata,
        integration_result['scgen'],
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True,
        embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
# Rename column
bench_result['scgen'].rename(columns = {0 : 'scgen'}, inplace = True)
bench_result['scgen']

In [ ]:
# Run mnn
integration_result['mnn'] = scib.integration.combat(adata, BATCH_KEY)
sc.pp.neighbors(integration_result['mnn'])
# Bench saucie
bench_result['mnn'] = scib.me.metrics(
        adata,
        integration_result['mnn'],
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True,
        embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
# Rename column
bench_result['mnn'].rename(columns = {0 : 'mnn'}, inplace = True)
bench_result['mnn']

In [ ]:
# Run scvi
integration_result['scvi'] = scib.integration.combat(adata, BATCH_KEY)
sc.pp.neighbors(integration_result['scvi'])
# Bench saucie
bench_result['scvi'] = scib.me.metrics(
        adata,
        integration_result['scvi'],
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True,
        embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
# Rename column
bench_result['scvi'].rename(columns = {0 : 'scvi'}, inplace = True)
bench_result['scvi']

In [ ]:
# Run scanvi
integration_result['scanvi'] = scib.integration.combat(adata, BATCH_KEY)
sc.pp.neighbors(integration_result['scanvi'])
# Bench saucie
bench_result['scanvi'] = scib.me.metrics(
        adata,
        integration_result['scanvi'],
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True,
        embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
# Rename column
bench_result['scanvi'].rename(columns = {0 : 'scanvi'}, inplace = True)
bench_result['scanvi']

In [ ]:
# Run desc
integration_result['desc'] = scib.integration.combat(adata, BATCH_KEY)
sc.pp.neighbors(integration_result['desc'])
# Bench saucie
bench_result['desc'] = scib.me.metrics(
        adata,
        integration_result['desc'],
        batch_key = BATCH_KEY,
        label_key = CELLTYPE_KEY,
        verbose = True,
        embed = 'X_pca',
        organism = 'human',
        **eval_metrics
)

In [ ]:
# Rename column
bench_result['desc'].rename(columns = {0 : 'desc'}, inplace = True)
bench_result['desc']

In [ ]:
merged_benchscores = pd.concat(bench_result.values(), axis = 1)

In [ ]:
merged_benchscores

In [ ]:
# Metrics for batch effect removal
batch_effect_removal = [
    'PCR_batch',
    'kBET',
    'iLISI',
    'graph_conn',
    'ASW_label/batch'
]

# Metrics for conservation of biological variance
biological_conservation_metrics = [
    'NMI_cluster/label',
    'ARI_cluster/label',
    'ASW_label',
    'isolated_label_F1',
    'isolated_label_silhouette',
    'cLISI',
    'hvg_overlap'
]


In [ ]:
total_scores = {}
for k in merged_benchscores.keys():
    batch_score = merged_benchscores[k].loc[batch_effect_removal].sum() / len(batch_effect_removal) * 0.4
    conservation_score = merged_benchscores[k].loc[biological_conservation_metrics].sum() / len(biological_conservation_metrics) * 0.6
    total_scores[k] = batch_score + conservation_score

In [ ]:
# Create a DataFrame from the total scores
totalscore_df = pd.DataFrame.from_dict({"Total score": total_scores}, orient="index")

# Use pd.concat to combine the DataFrames
merged_benchscores = pd.concat([merged_benchscores, totalscore_df])

# Transpose if needed
merged_benchscores = merged_benchscores.T


In [ ]:
# Move Total score column up
cols = merged_benchscores.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged_benchscores = merged_benchscores[cols]

In [ ]:
# Sort benchscore table by Total score
merged_benchscores.sort_values(by = 'Total score', ascending = False)

In [ ]:
merged_benchscores.to_csv('./data/PBMC/h5ad/all.csv')